Seq2Seq AutoEncoder flow using CarlaRecorded dataset:
=========================================================

In [1]:
# seeding

import pytorch_lightning as pl
import randomname

# get random version name before seeding
version = randomname.get_name()
seed = 22742

pl.seed_everything(seed, workers=True)

Global seed set to 22742


22742

Model to use:
-------------

In [2]:
from pedestrians_video_2_carla.data.carla.skeleton import CARLA_SKELETON
from pedestrians_video_2_carla.modules.movements.movements import MovementsModelOutputType
from pedestrians_video_2_carla.modules.movements.seq2seq import Seq2SeqFlatEmbeddings

model = Seq2SeqFlatEmbeddings(
    input_nodes = CARLA_SKELETON,
    output_nodes = CARLA_SKELETON,
    disable_lr_scheduler = True,
    lr = 1e-2,
    embeddings_size = None,
    embeddings_size_0 = 128,
    embeddings_size_1 = 64,
    embeddings_size_2 = 32,
    movements_output_type = MovementsModelOutputType.pose_2d,
    bidirectional = True,
)

Flow definition:
----------------

Also handles checkpoint loading if needed.

In [3]:
# ckpt_path='/runs/JAADOpenPoseDataModule/jaad_autoencoder_notebook/LinearAE2D/other-style/checkpoints/epoch=273-step=8493.ckpt'
ckpt_path = None

In [4]:
# get flow model (LitAutoencoderFlow)

from pedestrians_video_2_carla.modules.flow.autoencoder import LitAutoencoderFlow
from pedestrians_video_2_carla.data.base.base_transforms import BaseTransforms
from pedestrians_video_2_carla.loss import LossModes

if ckpt_path is not None:
    flow = LitAutoencoderFlow.load_from_checkpoint(
        checkpoint_path=ckpt_path,
        movements_model=model,
    )
else:
    flow = LitAutoencoderFlow(
        movements_model=model,
        transform=BaseTransforms.hips_neck_bbox,
        loss_modes=[LossModes.loc_2d],
    )

In [5]:
# DataModule to use

from pedestrians_video_2_carla.data.carla.carla_recorded_datamodule import CarlaRecordedDataModule

dm = CarlaRecordedDataModule(
    batch_size=256,
    clip_offset=4,
    clip_length=4,
    transforms=BaseTransforms.hips_neck_bbox,
    input_nodes=CARLA_SKELETON,
    skip_metadata=True,
    return_graph=model.needs_graph,
    return_confidence=model.needs_confidence,
    missing_point_probability=0.1,
    noise='gaussian',
    num_workers=0
)

Subsets dir: /outputs/CarlaRecordedDataModule/subsets/c0b8266e369b6797f04a82980768923c


In [6]:
# loggers and callbacks

import os

from pytorch_lightning.callbacks import LearningRateMonitor, ModelCheckpoint
from pytorch_lightning.loggers.tensorboard import TensorBoardLogger

from pedestrians_video_2_carla.loggers.pedestrian import PedestrianLogger
from pedestrians_video_2_carla.loggers.pedestrian.enums import PedestrianRenderers

logger = TensorBoardLogger(
    save_dir='/runs',
    name=os.path.join(
        dm.__class__.__name__,
        'carla_autoencoder_notebook',
        model.__class__.__name__,
    ),
    version=version,
    default_hp_metric=False,
)
pedestrian_logger = PedestrianLogger(
    save_dir=os.path.join(logger.log_dir, "videos"),
    name=logger.name,
    version=logger.version,
    movement_model_output_type=model.output_type,
    renderers=[PedestrianRenderers.source_videos, PedestrianRenderers.target_points, PedestrianRenderers.input_points, PedestrianRenderers.projection_points],
    source_videos_dir='/datasets/CARLA/BasicPedestriansCrossing',
    source_videos_overlay=True,
    video_saving_frequency_reduction=0  # this actually turns of the logger
)
checkpoint_callback = ModelCheckpoint(
    dirpath=os.path.join(logger.log_dir, "checkpoints"),
    monitor="val_loss/primary",
    mode="min",
    save_top_k=1,
)
lr_monitor = LearningRateMonitor(logging_interval="step")

/app/src/pedestrians_video_2_carla/loggers/pedestrian/pedestrian_logger.py:73: UserWarning: Video logging interval set to 0. Disabling video output.
  rank_zero_warn("Video logging interval set to 0. Disabling video output.")


In [7]:
# actual trainer

trainer = pl.Trainer(
    # gpus=1,
    # auto_select_gpus=True,
    log_every_n_steps=1,
    num_sanity_val_steps=1,
    check_val_every_n_epoch=1,
    max_epochs=10,
    deterministic=False,
    logger=[logger, pedestrian_logger],
    callbacks=[checkpoint_callback, lr_monitor],
)

GPU available: True, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
/venv/lib/python3.8/site-packages/pytorch_lightning/trainer/trainer.py:1579: UserWarning: GPU available but not used. Set the gpus flag in your trainer `Trainer(gpus=1)` or script `--gpus=1`.
  rank_zero_warn(
Running in fast_dev_run mode: will run a full train, val, test and prediction loop using 1 batch(es).


In [8]:
# train!

trainer.fit(model=flow, datamodule=dm, ckpt_path=ckpt_path)  # ckpt_path is needed to resume optimizer states etc.


  | Name             | Type                  | Params
-----------------------------------------------------------
0 | movements_model  | Seq2SeqFlatEmbeddings | 333 K 
1 | trajectory_model | ZeroTrajectory        | 0     
2 | metrics          | MetricCollection      | 0     
-----------------------------------------------------------
333 K     Trainable params
0         Non-trainable params
333 K     Total params
1.332     Total estimated model params size (MB)


------------------------------------------------------
Initial metrics:
------------------------------------------------------
initial/MJR: 0.099884033203125
initial/MSE: 87.10411834716797
initial/PCK@005: 0.5372126698493958
initial/PCKhn@01: 0.3494592607021332
------------------------------------------------------


/venv/lib/python3.8/site-packages/pytorch_lightning/trainer/data_loading.py:110: UserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 32 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/venv/lib/python3.8/site-packages/pytorch_lightning/trainer/data_loading.py:110: UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 32 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]